## Chapter 7 집계함수

집계는 무언가를 함께 모으는 행위이며 빅데이터 분석의 초석이다. 집계를 수행하려면 **키나 그룹**을 지정하고 하나 이상의 컬럼을 변환하는 방법을 지정하는 **집계함수**를 사용한다. 집계함수는 여러 입력값이 주어지면 그룹별로 결과를 생성한다

* 가장 간단한 형태의 그룹화는 select 구문에서 집계를 수행해 DataFrame의 전체 데이터를 요약하는 거다
* 'group by'는 하나 이상의 키를 지얼할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있다.
* '윈도우(window)'는 하나 이상의 키를 지얼할 수 있으며 값을 가진 컬ㄹ럼을 변환하기 위해 다른 집계 함수를 사용할 수 있다. 하지만 함수의 입력으로 사용할 로우는 현재 로우와 어느 정도 연관성이 있어야 한다.
* '그룹화 셋(grouping set)'은 서로 다른 레벨의 값을 집계할 때 사용한다. SQL, DataFrame의 롤압 그리고 큐브를 사용할 수 있다
* '롤업(rollup)'은 하나 이상의 키를 지정할 수 있다. 그리고 컬럼을 변환하는 데 다른 집계 함수를 사용하여 계층적으로 요약된 값을 구할 수 있다.
* '큐브(cube)'는 하나 이상의 키를 지정할 수 있으며 값을 가진 컬럼을 변환하기 위해 다른 집계 함수를 사용할 수 있다. 큐브는 모든 컬럼 조합에 대한 요약 값을 계산한다.

지정된 집계함수에 따라 그룹화된 결과는 RelationalGroupedDataset을 반환한다.

        어떤 결과를 만들어야 할지 정확히 파악해야 한다. 빅데이터를 사용해 연산을 수행하는 경우 질문에 대한 정확한 답을 얻기 위해서는 연산, 네트워크, 저장소 등 상당한 비용이 들 수 밖에 없다. 그러므로 수용 가능한 정도의 정확도에 맞춰 근사치를 계한하는 것이 비용을 고려했을 때 더 효율적이다. 

In [6]:
from pyspark.sql import SparkSession # SparkSession: 스파크 코드를 실행하기 위한 진입점

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
df = spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/Users/taewoong/Documents/coding/Spark_practice/data/retail-data/all/*.csv")\
    .coalesce(5)

In [5]:
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [6]:
df.createOrReplaceTempView('dfTable')

In [7]:
df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [8]:
df.count() ==541909

True

* count

In [10]:
from pyspark.sql.functions import count

df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



        null 값이 포함된 데이터의 레코드 수를 구할 때는 몇 가지 유의사항이 있다. count(*) 구문을 사용하면 null값을 가진 로우를 포함해 카운트한다. 하지만 count 함수에 특정 컬럼을 지정하면 null 값을 카운트 하지 않는다

* countDistinct : 전체 레코드 수가 아닌 고유 레코드 수를 구해야 한다면 countDistinct 함수를 사용한다

In [11]:
from pyspark.sql.functions import countDistinct

df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



* approx_count_distinct : 근사치 계산
        
      approx_count_distinct함수는 maximum estimation error라는 한 가지 파라미터를 더 사용한다. 예제에서는 큰 오류율 설정했기 때문에 기대치에서 크게 벗어나는 결과를 얻게 되지만, countDistinct함수보다 더 빠르게 결과를 반환한다. 이 함수의 성능은 대규모 데이터셋을 사용할 때 훨씬 더 좋아진다.

In [13]:
from pyspark.sql.functions import approx_count_distinct

df.select(approx_count_distinct("StockCode",0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



* first와 last : DataFrame의 첫 번째 값이나 마지막 값을 얻을 때 사용한다.

In [15]:
from pyspark.sql.functions import first, last

df.select(first("StockCode"),last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



* min, max 

In [16]:
from pyspark.sql.functions import min, max

df.select(min("Quantity"),max("Quantity")).show()


+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



* sum
* sumDistinct : 고유값의 합게
* avg

### 분산과 표준편차

분산은 평균과의 차이를 제곱한 결과의 평균이며 표준편차는 분산의 제곱근이다.<br>
스파크는 표본표준편차뿐만 아니라 모표준편차를 계산할 수 있다. 

In [17]:
from pyspark.sql.functions import var_pop,stddev_pop
from pyspark.sql.functions import var_samp,stddev_samp

In [18]:
df.select(var_pop("Quantity"),var_samp("Quantity"),stddev_pop("Quantity"),stddev_samp("Quantity")).show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
| 47559.3036466091|  47559.3914092988|  218.08095663447807|   218.08115785023426|
+-----------------+------------------+--------------------+---------------------+



### 비대칭도와 첨도

비대칭도와 첨도 모두 데이터의 변곡점을 측정하는 방법이다. 비대칭도는 데이터 평균의 비대칭정도를 측정하고, 첨도는 데이터 끝 부분을 측정한다. 비대칭도와 첨도는 확률변수의 확률분포로 데이터를 모델링할 때 측히 중요하다.

In [19]:
from pyspark.sql.functions import skewness, kurtosis

df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610529564|119768.05495534712|
+--------------------+------------------+



### 공분산과 상관관계

두 컬럼값 사이의 영향도를 비교하는 cov와 corr함수를 사용해 공분산과 상관관계를 계산한다.

In [20]:
from pyspark.sql.functions import corr, covar_pop, covar_samp

df.select(corr("InvoiceNo","Quantity"),covar_pop("InvoiceNo","Quantity"),covar_samp("InvoiceNo","Quantity")).show()

+-------------------------+------------------------------+-------------------------------+
|corr(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|
+-------------------------+------------------------------+-------------------------------+
|     4.912186085642775E-4|            1052.7260778754612|             1052.7280543915654|
+-------------------------+------------------------------+-------------------------------+



### 복합 데이터 타입의 집계

스파크는 수식을 이용한 집계뿐만 아니라 복합 데이터 타입을 사용해 집계를 수행할 수 있다. 예를 들어 특정 컬럼의 값을 리스트로 수집하거나 셋 데이터 타입으로 고윳값만 수집할 수 있다. 수집된 데이터는 파이프라인에서 다양한 프로그래밍 방식으로 다루거나 사용자 정의 함수를 사용해 전체 데이터에 접근할 수 있다.

In [21]:
from pyspark.sql.functions import collect_list, collect_set

In [22]:
df.agg(collect_set("Country"),collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



### 그룹화 

데이터 기반 **그룹**기반의 집계를 수행하는 경우가 더 많다. 데이터 그룹 기반의 집계는 단일 컬럼의 데이터를 그룹화하고 해당 그룹의 다른 여러 컬럼을 사용해서 계산하기 위해 카테고리형 데이터를 사용한다<br>
그룹화 작업은 하나 이상의 컬럼을 그룹화 하고 집계 연산을 수행하는 두 단계로 이뤄진다. 첫 번째 단계에서는 RelationalGroupedDataset이 반환되고, 두 번째 단계에서는 DataFrame이 반환된다. 그룹의 기준이 되는 컬럼을 여러 개 지정할 수 있다.

In [23]:
df.groupBy("InvoiceNo",'CustomerId').count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   543188|     12567|   63|
|   543590|     17377|   19|
|  C543757|     13115|    1|
|  C544318|     12989|    1|
|   544578|     12365|    1|
|   536596|      null|    6|
|   537252|      null|    1|
+---------+----------+-----+
only showing top 20 rows



### 표현식을 이용한 그룹화

* agg메서드 : 여러 집계 처리를 한 번에 지정할 수 있으며, 집계에 표현식을 사용할 수 있다. 또한 트랜스포메이션이 완료된 컬럼에 alias 메서드를 사용할 수 있다.

In [26]:
from pyspark.sql.functions import count, expr

In [27]:
df.groupBy("InvoiceNo").agg(count("Quantity").alias("quan"),expr("count(Quantity)")).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|  C542604|   8|              8|
+---------+----+---------------+
only showing top 20 rows



### 맵을 이용한 그룹화

컬럼을 키로, 수행할 집계 함수의 문자열을 값으로 하는 맵타입을 사용해 트랜스포메이션을 정의할 수 있습니다. 수행할 집계 함수를 한 줄로 작성하면 여러 컬럼명을 재사용할 수 있다.

In [28]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1